# Fundamental tensor decompositions.
### Last modification (11.04.2018)


In this tutorial we provide a theoretical backgound on the fundamental tensor
decompositions of multidimensional arrays and show how these data algorithms
can be used with [hottbox](https://github.com/hottbox/hottbox) through **TensorCPD**, **TensorTKD** and **TensorTT** classes.

More details on **CPD**, **HOSVD**, **HOOI** and **TTSVD** classes can be found on the [documentation page](https://hottbox.github.io/stable/api/hottbox.algorithms.decomposition).

> **Note:** this tutorial assumes that you are familiar with the basics of 
tensor algebra, tensor representaitons in different forms and the corresponding conventional 
notation. If you are new to these topics, check out our previous tutorials:
[tutorial_1](https://github.com/hottbox/hottbox-tutorials/blob/master/1_N-dimensional_arrays_and_Tensor_class.ipynb) and 
[tutorial_2](https://github.com/hottbox/hottbox-tutorials/blob/master/2_Efficient_representations_of_tensors.ipynb).

**Author:** Ilya Kisil - ilyakisil@gmail.com

In [1]:
import numpy as np
from hottbox.core.structures import Tensor
from hottbox.algorithms.decomposition import TTSVD, HOSVD, HOOI, CPD

In [2]:
I, J, K = 5, 6, 7

array_3d = np.arange(I * J * K).reshape((I, J, K)).astype(np.float)
# array_3d = np.random.rand(I * J * K).reshape((I, J, K)).astype(np.float)

tensor = Tensor(array_3d)

In [3]:
cpd = CPD()
kryskal_rank = (5,)

tensor_cpd = cpd.decompose(tensor, kryskal_rank)
type(tensor_cpd)

hottbox.core.structures.TensorCPD

In [4]:
np.sum(tensor.data - tensor_cpd.reconstruct.data)

2.217517913118704e-06

In [5]:
hosvd = HOSVD()
ml_rank = (2,3,4)

tensor_tkd = hosvd.decompose(tensor, ml_rank)
type(tensor_tkd)

/home/ik1614/anaconda2/envs/py36/lib/python3.6/site-packages/hottbox/algorithms/decomposition/base.py:79: RuntimeWarning: invalid value encountered in sqrt
  S = np.sqrt(S)


hottbox.core.structures.TensorTKD

In [6]:
np.sum(tensor.data - tensor_tkd.reconstruct.data)

-2.1126245502468058e-11

In [7]:
hooi = HOOI()
ml_rank = (2,3,4)

tensor_tkd = hosvd.decompose(tensor, ml_rank)
type(tensor_tkd)

/home/ik1614/anaconda2/envs/py36/lib/python3.6/site-packages/hottbox/algorithms/decomposition/base.py:79: RuntimeWarning: invalid value encountered in sqrt
  S = np.sqrt(S)


hottbox.core.structures.TensorTKD

In [8]:
np.sum(tensor.data - tensor_tkd.reconstruct.data)

-1.699809288433892e-11

In [9]:
tt = TTSVD()
tt_rank = (2,3)

tensor_tt = tt.decompose(tensor, tt_rank)
type(tensor_tt)

hottbox.core.structures.TensorTT

In [10]:
np.sum(tensor.data - tensor_tt.reconstruct.data)

-9.180048367208174e-12